In [7]:
using ITensors
using Random, Distributions
include("../BinarisedMPS/MPSLLOptimkit.jl")

ScoreMPS (generic function with 2 methods)

In [13]:
sites = siteinds("S=1/2", 10);
X = rand(5, 10)
y = rand([0,1], 5)
X_bin = BinariseDataset(X)
ps = GenerateAllProductStates(X_bin, y, "train", sites)

Initialising training states.


5-element Vector{PState}:
 PState(MPS
[1] ((dim=2|id=430|"S=1/2,Site,n=1"),)
[2] ((dim=2|id=499|"S=1/2,Site,n=2"),)
[3] ((dim=2|id=598|"S=1/2,Site,n=3"),)
[4] ((dim=2|id=360|"S=1/2,Site,n=4"),)
[5] ((dim=2|id=57|"S=1/2,Site,n=5"),)
[6] ((dim=2|id=162|"S=1/2,Site,n=6"),)
[7] ((dim=2|id=754|"S=1/2,Site,n=7"),)
[8] ((dim=2|id=830|"S=1/2,Site,n=8"),)
[9] ((dim=2|id=792|"S=1/2,Site,n=9"),)
[10] ((dim=2|id=580|"S=1/2,Site,n=10"),)
, 0, "train")
 PState(MPS
[1] ((dim=2|id=430|"S=1/2,Site,n=1"),)
[2] ((dim=2|id=499|"S=1/2,Site,n=2"),)
[3] ((dim=2|id=598|"S=1/2,Site,n=3"),)
[4] ((dim=2|id=360|"S=1/2,Site,n=4"),)
[5] ((dim=2|id=57|"S=1/2,Site,n=5"),)
[6] ((dim=2|id=162|"S=1/2,Site,n=6"),)
[7] ((dim=2|id=754|"S=1/2,Site,n=7"),)
[8] ((dim=2|id=830|"S=1/2,Site,n=8"),)
[9] ((dim=2|id=792|"S=1/2,Site,n=9"),)
[10] ((dim=2|id=580|"S=1/2,Site,n=10"),)
, 0, "train")
 PState(MPS
[1] ((dim=2|id=430|"S=1/2,Site,n=1"),)
[2] ((dim=2|id=499|"S=1/2,Site,n=2"),)
[3] ((dim=2|id=598|"S=1/2,Site,n=3"),)
[4] ((dim=2

In [17]:
ps[1].pstate

MPS
[1] ((dim=2|id=430|"S=1/2,Site,n=1"),)
[2] ((dim=2|id=499|"S=1/2,Site,n=2"),)
[3] ((dim=2|id=598|"S=1/2,Site,n=3"),)
[4] ((dim=2|id=360|"S=1/2,Site,n=4"),)
[5] ((dim=2|id=57|"S=1/2,Site,n=5"),)
[6] ((dim=2|id=162|"S=1/2,Site,n=6"),)
[7] ((dim=2|id=754|"S=1/2,Site,n=7"),)
[8] ((dim=2|id=830|"S=1/2,Site,n=8"),)
[9] ((dim=2|id=792|"S=1/2,Site,n=9"),)
[10] ((dim=2|id=580|"S=1/2,Site,n=10"),)


In [27]:
mps1 = randomMPS(sites; linkdims=2)

MPS
[1] ((dim=2|id=430|"S=1/2,Site,n=1"), (dim=2|id=380|"Link,l=1"))
[2] ((dim=2|id=380|"Link,l=1"), (dim=2|id=499|"S=1/2,Site,n=2"), (dim=2|id=240|"Link,l=2"))
[3] ((dim=2|id=240|"Link,l=2"), (dim=2|id=598|"S=1/2,Site,n=3"), (dim=2|id=746|"Link,l=3"))
[4] ((dim=2|id=746|"Link,l=3"), (dim=2|id=360|"S=1/2,Site,n=4"), (dim=2|id=472|"Link,l=4"))
[5] ((dim=2|id=472|"Link,l=4"), (dim=2|id=57|"S=1/2,Site,n=5"), (dim=2|id=168|"Link,l=5"))
[6] ((dim=2|id=168|"Link,l=5"), (dim=2|id=162|"S=1/2,Site,n=6"), (dim=2|id=251|"Link,l=6"))
[7] ((dim=2|id=251|"Link,l=6"), (dim=2|id=754|"S=1/2,Site,n=7"), (dim=2|id=236|"Link,l=7"))
[8] ((dim=2|id=236|"Link,l=7"), (dim=2|id=830|"S=1/2,Site,n=8"), (dim=2|id=369|"Link,l=8"))
[9] ((dim=2|id=369|"Link,l=8"), (dim=2|id=792|"S=1/2,Site,n=9"), (dim=2|id=43|"Link,l=9"))
[10] ((dim=2|id=43|"Link,l=9"), (dim=2|id=580|"S=1/2,Site,n=10"))


In [28]:
mps2 = randomMPS(sites; linkdims=2)

MPS
[1] ((dim=2|id=430|"S=1/2,Site,n=1"), (dim=2|id=554|"Link,l=1"))
[2] ((dim=2|id=554|"Link,l=1"), (dim=2|id=499|"S=1/2,Site,n=2"), (dim=2|id=806|"Link,l=2"))
[3] ((dim=2|id=806|"Link,l=2"), (dim=2|id=598|"S=1/2,Site,n=3"), (dim=2|id=297|"Link,l=3"))
[4] ((dim=2|id=297|"Link,l=3"), (dim=2|id=360|"S=1/2,Site,n=4"), (dim=2|id=108|"Link,l=4"))
[5] ((dim=2|id=108|"Link,l=4"), (dim=2|id=57|"S=1/2,Site,n=5"), (dim=2|id=252|"Link,l=5"))
[6] ((dim=2|id=252|"Link,l=5"), (dim=2|id=162|"S=1/2,Site,n=6"), (dim=2|id=662|"Link,l=6"))
[7] ((dim=2|id=662|"Link,l=6"), (dim=2|id=754|"S=1/2,Site,n=7"), (dim=2|id=520|"Link,l=7"))
[8] ((dim=2|id=520|"Link,l=7"), (dim=2|id=830|"S=1/2,Site,n=8"), (dim=2|id=563|"Link,l=8"))
[9] ((dim=2|id=563|"Link,l=8"), (dim=2|id=792|"S=1/2,Site,n=9"), (dim=2|id=477|"Link,l=9"))
[10] ((dim=2|id=477|"Link,l=9"), (dim=2|id=580|"S=1/2,Site,n=10"))


In [47]:
ψ1 = randomMPS(sites; linkdims=1)
ψ2 = randomMPS(sites; linkdims=1);
psis = [ψ1, ψ2]
ψ = ITensors.add(psis...; alg="directsum");
trunc

trunc (generic function with 65 methods)